In [1]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced
from imblearn.over_sampling import RandomOverSampler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import ClusterCentroids
from imblearn.combine import SMOTEENN



In [3]:
Path_1 = Path('/Users/ameerirfan/Desktop/FT-BTC-Activities/FT-BTC_Unit_11/LoanStats_2019Q1.csv')
df_1 = pd.read_csv(Path_1)
df_1.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,low_risk,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,Mar-2019,low_risk,n,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-2019,low_risk,n,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N


## Training & Testing


In [4]:
X = pd.get_dummies(df_1.drop('loan_status', axis=1))
y = df_1.loc[:, 'loan_status']

X.describe()

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,issue_d_Mar-2019,pymnt_plan_n,initial_list_status_f,initial_list_status_w,next_pymnt_d_Apr-2019,next_pymnt_d_May-2019,application_type_Individual,application_type_Joint App,hardship_flag_N,debt_settlement_flag_N
count,68817.000000,68817.000000,68817.000000,6.881700e+04,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,...,68817.000000,68817.0,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.0,68817.0
mean,16677.594562,0.127718,480.652863,8.821371e+04,21.778153,0.217766,0.497697,12.587340,0.126030,17604.142828,...,0.177238,1.0,0.123879,0.876121,0.383161,0.616839,0.860340,0.139660,1.0,1.0
std,10277.348590,0.048130,288.062432,1.155800e+05,20.199244,0.718367,0.758122,6.022869,0.336797,21835.880400,...,0.381873,0.0,0.329446,0.329446,0.486161,0.486161,0.346637,0.346637,0.0,0.0
min,1000.000000,0.060000,30.890000,4.000000e+01,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,...,0.000000,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,1.0
25%,9000.000000,0.088100,265.730000,5.000000e+04,13.890000,0.000000,0.000000,8.000000,0.000000,6293.000000,...,0.000000,1.0,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,1.0,1.0
50%,15000.000000,0.118000,404.560000,7.300000e+04,19.760000,0.000000,0.000000,11.000000,0.000000,12068.000000,...,0.000000,1.0,0.000000,1.000000,0.000000,1.000000,1.000000,0.000000,1.0,1.0
75%,24000.000000,0.155700,648.100000,1.040000e+05,26.660000,0.000000,1.000000,16.000000,0.000000,21735.000000,...,0.000000,1.0,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.0,1.0
max,40000.000000,0.308400,1676.230000,8.797500e+06,999.000000,18.000000,5.000000,72.000000,4.000000,587191.000000,...,1.000000,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.0,1.0


In [5]:
y.value_counts()

low_risk     68470
high_risk      347
Name: loan_status, dtype: int64

In [6]:
X_train, X_test, y_train, y_test= train_test_split(X, y, random_state=1, stratify=y)
X_train.shape

(51612, 95)

In [7]:
X_train

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,issue_d_Mar-2019,pymnt_plan_n,initial_list_status_f,initial_list_status_w,next_pymnt_d_Apr-2019,next_pymnt_d_May-2019,application_type_Individual,application_type_Joint App,hardship_flag_N,debt_settlement_flag_N
39692,10000.0,0.1131,328.86,49000.0,24.54,0.0,0.0,10.0,1.0,6580.0,...,0,1,0,1,0,1,1,0,1,1
37830,21000.0,0.1072,684.74,57400.0,34.52,0.0,0.0,12.0,0.0,18688.0,...,0,1,0,1,0,1,1,0,1,1
64531,35000.0,0.0881,1109.90,180000.0,3.01,0.0,2.0,7.0,0.0,836.0,...,0,1,0,1,0,1,1,0,1,1
28730,20000.0,0.0819,628.49,80000.0,26.51,0.0,0.0,15.0,0.0,2615.0,...,0,1,0,1,0,1,1,0,1,1
35102,12600.0,0.1356,427.96,71000.0,22.38,1.0,2.0,7.0,0.0,9073.0,...,0,1,0,1,0,1,1,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18872,35000.0,0.0881,1109.90,400000.0,13.79,0.0,1.0,17.0,1.0,7810.0,...,0,1,0,1,1,0,1,0,1,1
6415,16800.0,0.0881,532.76,31000.0,20.48,0.0,0.0,3.0,0.0,1867.0,...,1,1,0,1,0,1,1,0,1,1
12632,25000.0,0.1102,818.71,90000.0,15.27,1.0,0.0,18.0,0.0,26106.0,...,0,1,0,1,1,0,1,0,1,1
24022,14000.0,0.1072,456.49,180000.0,12.35,2.0,0.0,10.0,0.0,3601.0,...,0,1,0,1,0,1,1,0,1,1


In [8]:
scaler = StandardScaler()
X_scaler = scaler.fit(X_train)

X_scaled_train = X_scaler.transform(X_train)
X_scaled_test = X_scaler.transform(X_test)

In [9]:
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_train, y_train)

LogisticRegression(random_state=1)

In [10]:
y_predictor = model.predict(X_test)
balanced_accuracy_score(y_test, y_predictor)

0.5057471264367817

In [11]:
confusion_matrix(y_test, y_predictor)

array([[    1,    86],
       [    0, 17118]])

In [12]:
print(classification_report_imbalanced(y_test, y_predictor))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       1.00      0.01      1.00      0.02      0.11      0.01        87
   low_risk       1.00      1.00      0.01      1.00      0.11      0.01     17118

avg / total       1.00      1.00      0.02      0.99      0.11      0.01     17205



## OVERSAMPLING - MODEL 1 & 2


In [13]:
ROSampler = RandomOverSampler(random_state=1)
resampled_X, resampled_Y = ROSampler.fit_resample(X_scaled_train, y_train)


Counter(resampled_Y)

Counter({'low_risk': 51352, 'high_risk': 51352})

In [14]:
over_model_1 = LogisticRegression(solver='lbfgs', random_state=1)
over_model_1.fit(resampled_X, resampled_Y)

LogisticRegression(random_state=1)

In [15]:
y_predictions = over_model_1.predict(X_scaled_test)
balanced_accuracy_score(y_test, y_predictions)

0.7856360112968401

In [16]:
confusion_matrix(y_test, y_predictions)

array([[   62,    25],
       [ 2420, 14698]])

In [17]:
print(classification_report_imbalanced(y_test, y_predictions))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.02      0.71      0.86      0.05      0.78      0.60        87
   low_risk       1.00      0.86      0.71      0.92      0.78      0.62     17118

avg / total       0.99      0.86      0.71      0.92      0.78      0.62     17205



In [18]:
smote = SMOTE(random_state=1, sampling_strategy=1.0)
resampled_X, resampled_Y = smote.fit_resample(X_scaled_train, y_train)

Counter(resampled_Y)

Counter({'low_risk': 51352, 'high_risk': 51352})

In [19]:
smote_model_2 = LogisticRegression(solver='lbfgs', random_state=1)
smote_model_2.fit(resampled_X, resampled_Y)

LogisticRegression(random_state=1)

In [20]:
y_predictions = smote_model_2.predict(X_scaled_test)

balanced_accuracy_score(y_test, y_predictions)

0.7966770207605626

In [21]:
confusion_matrix(y_test, y_predictions)

array([[   62,    25],
       [ 2042, 15076]])

In [22]:
print(classification_report_imbalanced(y_test, y_predictions))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.03      0.71      0.88      0.06      0.79      0.62        87
   low_risk       1.00      0.88      0.71      0.94      0.79      0.64     17118

avg / total       0.99      0.88      0.71      0.93      0.79      0.64     17205



## UNDERSAMPLING - MODEL 3

In [23]:
ClustCent = ClusterCentroids(random_state=1)
resampled_X, resampled_Y = ClustCent.fit_resample(X_scaled_train, y_train)

Counter(resampled_Y)

Counter({'high_risk': 260, 'low_risk': 260})

In [24]:
cluster_model_3 = LogisticRegression(solver='lbfgs', random_state=1)
cluster_model_3.fit(resampled_X, resampled_Y)

LogisticRegression(random_state=1)

In [25]:
y_predictions = cluster_model_3.predict(X_scaled_test)

balanced_accuracy_score(y_test, y_predictions)

0.7742313998976678

In [26]:
confusion_matrix(y_test, y_predictions)

array([[   68,    19],
       [ 3991, 13127]])

In [27]:
print(classification_report_imbalanced(y_test, y_predictions))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.02      0.78      0.77      0.03      0.77      0.60        87
   low_risk       1.00      0.77      0.78      0.87      0.77      0.60     17118

avg / total       0.99      0.77      0.78      0.86      0.77      0.60     17205



## COMBINATION SAMPLING - MODEL 4

In [28]:
SMOTEENN = SMOTEENN(random_state=1)
resampled_X, resampled_Y = SMOTEENN.fit_resample(X_scaled_train, y_train)

Counter(resampled_Y)

Counter({'high_risk': 51352, 'low_risk': 47520})

In [29]:
smoteenn_model_4 = LogisticRegression(solver='lbfgs', random_state=1)
smoteenn_model_4.fit(resampled_X, resampled_Y)

LogisticRegression(random_state=1)

In [31]:
y_predictions = smoteenn_model_4.predict(X_scaled_test)

balanced_accuracy_score(y_test, y_predictions)

0.7975462408998795

In [32]:
confusion_matrix(y_test, y_predictions)


array([[   63,    24],
       [ 2209, 14909]])

In [34]:
print(classification_report_imbalanced(y_test, y_predictions))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.03      0.72      0.87      0.05      0.79      0.62        87
   low_risk       1.00      0.87      0.72      0.93      0.79      0.64     17118

avg / total       0.99      0.87      0.72      0.93      0.79      0.64     17205



# FINAL QUESTIONS!


### 1. Which model had the best balanced accuracy score?

###### The model that had the best balanced accuracy score was the combination sampling model with a score of 79.7546%

### 2. Which model had the best recall score?

###### The model with the best recall score overall was the SMOTE Oversampling model with a recall score of 0.88

### 3. Which model had the best geometric mean score?

###### The model that had the best geometric mean score was the combination sampling model with a score of 0.79